# Light. Version 1

In [0]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential # class of model
from tensorflow.keras.layers import Dense # fullconnected layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image # for add image
import numpy as np
import pandas as pd
from google.colab import output
import seaborn as sns
import pylab
from google.colab import files
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [0]:
model = Sequential() # make neuralnetwork
model.add(Dense(800, input_dim = 784, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 800)               628000    
_________________________________________________________________
dense_11 (Dense)             (None, 400)               320400    
_________________________________________________________________
dense_12 (Dense)             (None, 10)                4010      
Total params: 952,410
Trainable params: 952,410
Non-trainable params: 0
_________________________________________________________________


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
print(x_test.shape)

(10000, 784)


In [0]:
# normalize data
x_train = x_train.astype('float32')
x_train = x_train / 255
x_test = x_test.astype('float32')
x_test = x_test / 255

In [0]:
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

In [0]:
model.fit(x_train, y_train, batch_size=128, epochs=15, verbose=1)
# verbose = visualation progress

Epoch 1/15
469/469 [==============================] - 7s 16ms/step - loss: 0.2061 - accuracy: 0.9389
Epoch 2/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0767 - accuracy: 0.9766
Epoch 3/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0485 - accuracy: 0.9840
Epoch 4/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0356 - accuracy: 0.9886
Epoch 5/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0266 - accuracy: 0.9915
Epoch 6/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0198 - accuracy: 0.9935
Epoch 7/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0202 - accuracy: 0.9933
Epoch 8/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0153 - accuracy: 0.9950
Epoch 9/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0126 - accuracy: 0.9960
Epoch 10/15
469/469 [==============================] - 7s 15ms/step - loss: 0.0127 - accura

In [0]:
model.save_weights('model_base.h5')
model.load_weights('model_base.h5')

In [0]:
res_base = model.evaluate(x_test, y_test, verbose=1)
print(res_base)

313/313 [==============================] - 1s 3ms/step - loss: 0.1169 - accuracy: 0.9777
[0.11694846302270889, 0.9776999950408936]


# Light. Version 2

In [0]:
# repeat prepare data for convenience and stability
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_train = x_train / 255
x_test = x_test.astype('float32')
x_test = x_test / 255
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

In [0]:
var_neu = pd.DataFrame(columns = ['n_hide_layer', 'act_func', 'batch_size', 'accuracy'])
'''
Для удобного анализа сделаем таблицу точности наших нейронных сетей.
n_hide_layer - количество нейронов в единственном скрытом слое: 10, 100, 400 или 5000;
(первый слой 800 нейронов, последний - 10)
act_func - активационная функция в скрытом слое: relu или linear;
batch_size - 1, 10, 128 или вся база (60000);
'''
var_neu

,n_hide_layer,act_func,batch_size,accuracy


In [0]:
# checking if gpu works
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
list_nh = [10, 100, 400, 5000]
list_af = ['relu', 'linear']
list_bs = [1, 10, 128, 60000]
i = 0 # not smart method for count index =)
for nh in list_nh:
  for af in list_af:
    for bs in list_bs:
      model = Sequential()
      model.add(Dense(800, input_dim = 784, activation='relu'))
      model.add(Dense(nh, activation=af))
      model.add(Dense(10, activation='softmax'))
      model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      model.fit(x_train, y_train, batch_size=bs, epochs=15, verbose=0)
      res_base = model.evaluate(x_test, y_test, verbose=0)
      var_neu.loc[i] = [nh, af, bs, res_base[1]]
      i += 1
  # Play an audio beep and text-information.
  print('Success nh:', nh)
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [0]:
var_neu.to_csv('var_neu.csv')
files.download("var_neu.csv")

In [0]:
var_neu = pd.read_csv('var_neu.csv', index_col=0)

In [0]:
var_neu.sort_values('accuracy', ascending=False)

,n_hide_layer,act_func,batch_size,accuracy
6,10,linear,128,0.9831
26,5000,relu,128,0.9826
14,100,linear,128,0.9820
22,400,linear,128,0.9818
10,100,relu,128,0.9816
13,100,linear,10,0.9811
2,10,relu,128,0.9810
5,10,linear,10,0.9809
18,400,relu,128,0.9803
9,100,relu,10,0.9789


## Выводы.

От количества нейронов время работы не так зависит, как от batch_size. Чем он меньше, тем дольше будет считаться нейронка.

Этот же показатель оказался наиболее важным для качества нейросети, но здесь нету прямой корреляции, нельзя просто сделать их поменьше или побольше. Нужно подбирать значение. Но однозначно можно сказать, что брать всю выборку 60000 не есть гуд идея. 

Однако, я полагаю, что ключевой фактор может быть не столько сам параметр, сколько количество пересчетов весов, которые следуют за ним, если не менять количество эпох. Чтобы проверить этот факт, можно сделать тест, который будет менять batch_size и epochs таким образом, чтобы количество пересчетов оставалось равным. Извиняюсь если формулировки не совсем точные, не знаю как правильно называется перераспределение весов.

Также интересным и неожиданным обстоятельством является преобладание результатов линейной функции над релу. 3 из 4 первых результата поставлены на моделях с 'linear'.

Не знаю насколько это верное решение, я же только учусь. Но я предпочту для дальнейших исследований и экспериментов взять третью модель, а не первую. Дело в том, что во-первых разница между ними всего на 1 тысячную процента! Нейросеть номер один работает на 1\100 000 лучше чем нейросеть три! У нас же тут не Kaggle, чтобы бороться за такие цифры) при этом тестовая выборка, это всего-лишь выборка, конечная цель, чтобы нейросеть лучше определяла новые рукописные цифры. А количество нейронов 100, мне кажется дает больше возможносетй нейросети, чем 10. Но это возможно ошибочные умозаключения. Всего не перепроверишь за время отведенное под домашнюю работу.



In [21]:
count = int(60000/128*15)
list_bs = [128, 128*6, 128*30, 60000]
for bs in list_bs:
  model = Sequential()
  model.add(Dense(800, input_dim = 784, activation='relu'))
  model.add(Dense(100, activation='linear'))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(x_train, y_train, batch_size=bs, epochs=int(count/(60000/bs)), verbose=0)
  res_base = model.evaluate(x_test, y_test, verbose=0)
  print(f'nh = 100, af = "linear", bs = {bs}, epochs = {count/(60000/bs)}, accuracy = {res_base[1]}')
print('Success!')

nh = 100, af = "linear", bs = 128, epochs = 14.999466666666667, accuracy = 0.9751999974250793
nh = 100, af = "linear", bs = 768, epochs = 89.9968, accuracy = 0.9847000241279602
nh = 100, af = "linear", bs = 3840, epochs = 449.984, accuracy = 0.9824000000953674
nh = 100, af = "linear", bs = 60000, epochs = 7031.0, accuracy = 0.9799000024795532
Success!


**Полагаю теорию можно считать успешной, результат улучшен, хоть и не значительно.**

Мы улучшили лучший результат с прошлой таблицы на 1.5 тысячных процента) На кагле это бы сдвинуло нас на несколько десятков мест вперед! =) Не так уж плохо, для первого урока)))